# Pangraph - Node k-mer analysis & seq redundancy comparison

### Import Statements

In [1]:
import numpy as np
import pandas as pd
import scipy.stats

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
import ast

In [3]:
# https://bioframe.readthedocs.io/en/latest/guide-intervalops.html
import bioframe as bf


In [4]:
from Bio import SeqIO


In [5]:
#import json

In [6]:
import gfapy

#### Pandas Viewing Settings

In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
pd.set_option('max_colwidth', 400)

## Define useful Kmer analysis functions

In [9]:
import screed

In [10]:
import mmh3

In [11]:
def build_kmers(sequence, ksize):
    kmers = []
    n_kmers = len(sequence) - ksize + 1
    
    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)
        
    return kmers

In [12]:
#import screed a library for reading in FASTA/FASTQ

def read_kmers_from_file(filename, ksize):
    all_kmers = []
    for record in screed.open(filename):
        sequence = record.sequence
        
        kmers = build_kmers(sequence, ksize)
        all_kmers += kmers

    return all_kmers

In [13]:
def hash_kmer(kmer):
    # calculate the reverse complement
    rc_kmer = screed.rc(kmer)
    
    # determine whether original k-mer or reverse complement is lesser
    if kmer < rc_kmer:
        canonical_kmer = kmer
    else:
        canonical_kmer = rc_kmer
        
    # calculate murmurhash using a hash seed of 42
    hash = mmh3.hash64(canonical_kmer, 42)[0]
    if hash < 0: hash += 2**64

    return hash

In [14]:
# def hash_kmers(kmers):
#     hashes = []
#     for kmer in kmers:
#         hashes.append(hash_kmer(kmer))
#     return hashes

def hash_kmers_ToSet(kmers):
    hashes = set()
    for kmer in kmers:
        hashes.add(hash_kmer(kmer))
    return hashes

In [15]:

def jaccard_containment_FromSets(a, b):
    '''
    This function returns the Jaccard Containment between sets a and b.
    '''
    
    intersection = len(a.intersection(b))
    
    return intersection / len(a)

def jaccard_similarity_FromSets(a, b):
    '''
    This function returns the Jaccard Similarity between sets a and b.
    '''
    intersection = len(a.intersection(b))
    union = len(a.union(b))
    
    return intersection / union


In [16]:
def getAllHash_ExceptTargets_Set_V2(dictOfHashes, targetsToRemove):
    # Convert targetsToRemove to a set for faster lookup
    targetsToRemoveSet = set(targetsToRemove)

    # Use set comprehension for more efficient construction of the result set
    return {hash for seqID, seqInfoDict in dictOfHashes.items() if seqID not in targetsToRemoveSet
            for hash in seqInfoDict["Kmer_Hashes_Set"]}

In [17]:
def getAllHash_InTargetSeqs_Set(dictOfHashes, targetsToKeep):
    
    all_Hashes_InTarget = set()

    for i_SeqID, i_SeqInfoDict in dictOfHashes.items():
        
        i_Hashes = i_SeqInfoDict["Kmer_Hashes_Set"]
        
        if i_SeqID not in targetsToKeep:
            all_Hashes_InTarget.update(i_Hashes) 
    
    return all_Hashes_InTarget

In [18]:
#%reload_ext autoreload
#%autoreload 2

### import panqc toolkit functions
#from panqc.kmerlib import hash_kmers_ToSet, jaccard_similarity_FromSets, jaccard_containment_FromSets


# Define pangraph processing functions

In [19]:
def parse_pangraph_fasta(fasta_path):
    """
    Parses the FASTA file to extract node sequences.
    
    Args:
        fasta_path (str): Path to the FASTA file.
    
    Returns:
        dict: A dictionary with node IDs as keys and sequences as values.
    """
    node_seqs = {}
    for record in tqdm(SeqIO.parse(fasta_path, "fasta"), desc="Parsing FASTA"):
        node_seqs[record.name] = str(record.seq).upper()
    return node_seqs

In [20]:
def parse_pangraph_gfa(gfa_path, node_seqs, k_size, num_assemblies):
    """
    Parses the GFA file and adds sequence information, k-mer hashes, and statistics.
    
    Args:
        gfa_path (str): Path to the GFA file.
        node_seqs (dict): Dictionary of node sequences.
        k_size (int): Size of k-mers for hashing.
        num_assemblies (int): Total number of assemblies in Pangraph.
    
    Returns:
        dict: A dictionary containing node information and statistics.
    """
    gfa = gfapy.Gfa.from_file(gfa_path, version="gfa1", vlevel=0)
    node_info = {}
    
    # Process S-lines
    for line in tqdm(gfa.lines, desc="Processing GFA S-lines"):
        if str(line).startswith("S"):
            parts = str(line).split("\t")
            node_id = parts[1]
            seq = node_seqs[node_id]
            seq_len = len(seq)
            rc = int(parts[4].split(":i:")[-1])
            
            # Generate k-mers and hashes
            kmers = build_kmers(seq, k_size)
            hashes = hash_kmers_ToSet(kmers)
            
            node_info[node_id] = {
                "Len": seq_len,
                "N_Asms": 0,
                "N_Occurrences": 0,
                "Kmers": kmers,
                "Kmer_Hashes_Set": hashes,
                "RC": rc
            }
    
    # Process P-lines
    for line in tqdm(gfa.lines, desc="Processing GFA P-lines"):
        if str(line).startswith("P"):
            parts = str(line).split("\t")
            node_path = parts[2].split(",")
            unique_nodes = {node[:-1] for node in node_path}
            
            for node_id in unique_nodes:
                node_info[node_id]["N_Asms"] += 1
            for node in node_path:
                node_id = node[:-1]
                node_info[node_id]["N_Occurrences"] += 1
    
    return node_info

In [21]:
def classify_pangraph_nodes(node_info, num_input_assemblies):
    """
    Classifies nodes as Core or SV based on the number of assemblies.
    
    Args:
        node_info (dict): Node information and statistics.
        num_assemblies (int): Total number of assemblies in Pangraph.
    
    Returns:
        tuple: Lists of core nodes and SV nodes.
    """
    core_nodes = []
    sv_nodes = []
    
    for node_id, info in node_info.items():
        if info["N_Asms"] > num_input_assemblies:
            raise ValueError(f"Node {node_id} has too many counts: {info['N_Asms']}")
        
        if info["N_Asms"] == num_input_assemblies:
            info["Type"] = "Core"
            core_nodes.append(node_id)
        else:
            info["Type"] = "SV"
            sv_nodes.append(node_id)
    
    return core_nodes, sv_nodes

In [22]:

def pangraph_pairwise_kmerset_comparison(node_info):
    """
    Perform pairwise analysis of k-mer content between all nodes.
    
    Args:
        node_info (dict): Dictionary containing node information.
    
    Returns:
        pd.DataFrame: DataFrame containing pairwise comparisons with Jaccard metrics.
    """
    pairwise_results = []

    AllNodeIDs = list(node_info.keys())
    
    for node_1 in tqdm(AllNodeIDs, desc="Running pairwise comparisons of k-mer content"):
        for node_2 in AllNodeIDs:
            # Skip comparisons if k-mer hashes are missing
            if "Kmer_Hashes_Set" not in node_info[node_1] or "Kmer_Hashes_Set" not in node_info[node_2]:
                continue

            hashes_1 = node_info[node_1]["Kmer_Hashes_Set"]
            hashes_2 = node_info[node_2]["Kmer_Hashes_Set"]

            # Skip if no overlap of k-mers
            if not (hashes_1 & hashes_2):
                continue

            # Calculate Jaccard Similarity
            js = jaccard_similarity_FromSets(hashes_1, hashes_2)

            # Skip if Jaccard Similarity is 0
            if js == 0:
                continue

            # Calculate Jaccard Containment
            jc = jaccard_containment_FromSets(hashes_1, hashes_2)

            len_1 = node_info[node_1]["Len"]
            len_2 = node_info[node_2]["Len"]

            # Append results as a tuple
            pairwise_results.append((node_1, node_2, len_1, len_2, js, jc))
    
    # Convert results to a DataFrame
    columns = ["RecordID_1", "RecordID_2", "Record1_Len", "Record2_Len", "JaccardSim", "JaccardContain"]
    AvA_results_df = pd.DataFrame(pairwise_results, columns=columns)
    
    # Exclude self-comparisons
    AvA_results_df = AvA_results_df.query("RecordID_1 != RecordID_2")
    
    return AvA_results_df


In [23]:
def compare_kmers_to_profiles_with_max_jc(node_info, sv_node_ids, ref_kmer_sets, ava_nodes_df):
    """
    Compares k-mer content of graph nodes to predefined k-mer profiles and calculates MaxJC.
    
    Args:
        node_info (dict): Dictionary containing node information.
        sv_node_ids (list): List of SV node IDs.
        kmer_sets (dict): Dictionary of predefined k-mer sets (e.g., H37Rv, IS6110, etc.).
        jaccard_containment_fn (function): Function to calculate Jaccard Containment.
        ava_nodes_df (pd.DataFrame): DataFrame of pairwise comparisons with Jaccard metrics.
    
    Returns:
        pd.DataFrame: DataFrame containing comparison results and MaxJC values.
    """

    AllNodeIDs = list(node_info.keys())
    
    # Step 1: Create a dict mapping NodeID to the maximum Jaccard Containment (MaxJC)
    max_jc_dict = ava_nodes_df.groupby("RecordID_1")["JaccardContain"].max().to_dict()
    
    results = []

    for node_id, info in tqdm(node_info.items(), desc="Comparing node to reference k-mer profiles"):
        record_hashes = info.get("Kmer_Hashes_Set", set())
        len_seq = info.get("Len", 0)
        n_asms = info.get("N_Asms", 0)
        n_occurrences = info.get("N_Occurrences", 0)
        
        # Initialize results with zero if no k-mers exist
        jc_results = {key: 0 for key in ref_kmer_sets.keys()}
        if record_hashes:
            for key, kmer_set in ref_kmer_sets.items():
                jc_results[key] = jaccard_containment_FromSets(record_hashes, kmer_set)
        else:
            if len_seq >= 31:
                print(f"No k-mers were produced for segment: {node_id}")
        
        # Determine if the node is an SV node
        is_sv_node = node_id in sv_node_ids

        # Get MaxJC value for the node
        max_jc_to_other_node = max_jc_dict.get(node_id, 0)

        # Collect results for this node
        row = (
            node_id, len_seq, n_asms, n_occurrences,
            jc_results.get("H37Rv", 0),
            jc_results.get("IS6110", 0),
            jc_results.get("Rv_InsSeqAndPhages", 0),
            is_sv_node, max_jc_to_other_node
        )
        results.append(row)
    
    # Convert results to a DataFrame
    columns = [
        "NodeID", "SeqLength", "N_Asms", "N_Occurrences",
        "Jaccard_Cont_WiRv", "Jaccard_Cont_WiIS6110",
        "Jaccard_Cont_WiRv_InsSeqAndPhages", "IsSVNode", "MaxJC_ToOtherNode"
    ]
    results_df = pd.DataFrame(results, columns=columns)
    
    return results_df


# Define function for summarizing pangraph stats regarding SV & Core Nodes

In [24]:
def summarize_Pangenome_Node_Stats(node_summary_df):
    """
    Summarizes key statistics from the pan-genome graph node summary DataFrame.
    
    Args:
        node_summary_df (pd.DataFrame): DataFrame containing node summaries.
    
    Returns:
        dict: A dictionary summarizing key statistics.
    """
    stats = {}

    # Core Nodes
    core_nodes_all = node_summary_df.query("IsSVNode == False")
    core_nodes_pass = core_nodes_all.query("SeqLength >= 31")
    core_nodes_sub31bp = core_nodes_all.query("SeqLength < 31")

    # SV Nodes
    sv_nodes_all = node_summary_df.query("IsSVNode == True")
    sv_nodes_pass = sv_nodes_all.query("SeqLength >= 31")
    sv_nodes_sub31bp = sv_nodes_all.query("SeqLength < 31")

    # SV Nodes by MaxJC threshold
    sv_pass_UnqInGraph = sv_nodes_pass.query("MaxJC_ToOtherNode < 0.05")
    sv_pass_RepeatInGraph = sv_nodes_pass.query("MaxJC_ToOtherNode >= 0.05")

    # SV Nodes by Rv match threshold
    sv_pass_UnqToH37Rv = sv_nodes_pass.query("Jaccard_Cont_WiRv < 0.05")
    sv_pass_InH37Rv = sv_nodes_pass.query("Jaccard_Cont_WiRv >= 0.05")
    

    stats["N_CoreNodes"] = core_nodes_all.shape[0]
    stats["SeqLength_CoreNodes"] = core_nodes_all["SeqLength"].sum()
    
    stats["N_SV_All"] = sv_nodes_all.shape[0]
    stats["SeqLength_SV_All"] = sv_nodes_all["SeqLength"].sum()

    stats["N_SV_UnqInGraph"] = sv_pass_UnqInGraph.shape[0]
    stats["SeqLength_SV_UnqInGraph"] = sv_pass_UnqInGraph["SeqLength"].sum()

    stats["N_SV_RepeatInGraph"] = sv_pass_RepeatInGraph.shape[0]
    stats["SeqLength_SV_RepeatInGraph"] = sv_pass_RepeatInGraph["SeqLength"].sum()

    stats["N_SV_UnqToH37Rv"] = sv_pass_UnqToH37Rv.shape[0]
    stats["SeqLength_SV_UnqToH37Rv"] = sv_pass_UnqToH37Rv["SeqLength"].sum()

    return stats


# Part 1: Parse sample metadata & preprocessed genome info/results

In [25]:
Repo_DataDir = "../../Data"
InputAsmPath_Dir = f"{Repo_DataDir}/231121.InputAsmTSVs.MtbSetV3.151CI"

MtbSetV3_151CI_InputAsmPATHs_TSV = f"{InputAsmPath_Dir}/231121.MtbSetV3.151CI.HybridAndSRAsm.FAPATHs.V1.tsv"
MtbSetV3_151CI_AsmSumm_TSV = f"{InputAsmPath_Dir}/231121.MtbSetV3.151CI.HybridAsm.AsmSummary.V2.tsv"   


## Parse sample Metadata (N = 151)

In [26]:
WGA151CI_AsmSummary_DF = pd.read_csv(MtbSetV3_151CI_AsmSumm_TSV, sep = "\t")

SampleIDs_151CI_SOI = list( WGA151CI_AsmSummary_DF["SampleID"].values )
WGA151CI_SampleIDs = SampleIDs_151CI_SOI

ID_To_PrimLineage_Dict = dict(WGA151CI_AsmSummary_DF[['SampleID', 'PrimaryLineage']].values)
ID_To_SubLineage_Dict = dict( WGA151CI_AsmSummary_DF[["SampleID", "Lineage"]].values)
ID_To_Dataset_Dict = dict(WGA151CI_AsmSummary_DF[['SampleID', 'Dataset_Tag']].values) 


## Parse FASTA paths for 151 Mtb assemblies

In [27]:
WGA151CI_Asm_Path_DF = pd.read_csv(MtbSetV3_151CI_InputAsmPATHs_TSV, sep = "\t")
WGA151CI_Asm_Path_DF.shape

(151, 4)

In [28]:
WGA151CI_Asm_Path_DF.head(4)

,SampleID,Dataset_Tag,Genome_ASM_PATH,ShortRead_Genome_ASM_PATH
0,N0072,ChinerOms_2019,/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/ChinerOms_2019/N0072.LR.Asm.fasta,/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/ChinerOms_2019/N0072.SR.Asm.fasta
1,N0153,ChinerOms_2019,/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/ChinerOms_2019/N0153.LR.Asm.fasta,/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/ChinerOms_2019/N0153.SR.Asm.fasta
2,TB3113,TB_Portals_24CI_R1,/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/TB_Portals_24CI_R1/TB3113.LR.Asm.fasta,/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/TB_Portals_24CI_R1/TB3113.SR.Asm.fasta
3,TB1236,TB_Portals_24CI_R1,/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/TB_Portals_24CI_R1/TB1236.LR.Asm.fasta,/n/data1/hms/dbmi/farhat/mm774/Projects/231121.MtbSetV3.151CI.CompleteAndSR.Asms/TB_Portals_24CI_R1/TB1236.SR.Asm.fasta


In [29]:
SampleID_To_LRAsmFA_PATH_Dict = dict(WGA151CI_Asm_Path_DF[['SampleID', 'Genome_ASM_PATH']].values)

## Import/parse processed H37rv genome annotations

In [30]:
RepoRef_Dir = "../../References"

AnnotatedGenes_And_IntergenicRegions_RepoRef_Dir = f"{RepoRef_Dir}/201027_H37rv_AnnotatedGenes_And_IntergenicRegions"
H37Rv_GenomeAnnotations_Genes_TSV = f"{AnnotatedGenes_And_IntergenicRegions_RepoRef_Dir}/H37Rv_GenomeAnnotations.Genes.tsv"

## H37Rv Gene Annotations TSV
H37Rv_GenomeAnno_Genes_DF = pd.read_csv(H37Rv_GenomeAnnotations_Genes_TSV, sep = "\t")
H37Rv_GeneInfo_Subset_DF = H37Rv_GenomeAnno_Genes_DF[["H37rv_GeneID", "Symbol", "Feature", "Functional_Category", "Is_Pseudogene", "Product", "PEandPPE_Subfamily", "ExcludedGroup_Category"]]

RvID_To_Symbol_Dict = dict(H37Rv_GeneInfo_Subset_DF[['H37rv_GeneID', 'Symbol']].values)
Symbol_To_FuncCat_Dict = dict(H37Rv_GeneInfo_Subset_DF[['Symbol', 'Functional_Category']].values)


In [31]:
H37Rv_GenomeAnno_Genes_DF.head(3)

,Chrom,Start,End,Strand,H37rv_GeneID,Symbol,Feature,Functional_Category,Is_Pseudogene,Product,PEandPPE_Subfamily,ExcludedGroup_Category
0,NC_000962.3,0,1524,+,Rv0001,dnaA,CDS,information pathways,No,Chromosomal replication initiator protein DnaA,None,NotExcluded
1,NC_000962.3,2051,3260,+,Rv0002,dnaN,CDS,information pathways,No,DNA polymerase III (beta chain) DnaN (DNA nucleotidyltransferase),None,NotExcluded
2,NC_000962.3,3279,4437,+,Rv0003,recF,CDS,information pathways,No,DNA replication and repair protein RecF (single-strand DNA binding protein),None,NotExcluded


# Part 2: Generate reference k-mer sets (ie H37Rv, IS6110, Phages + ISs)  

### Define k-mer size for entire analysis

In [32]:
k_size = 31

## Generate k-mer info for H37Rv and a representative IS6110 sequence 

In [33]:
Mtb_RefDir="/n/data1/hms/dbmi/farhat/mm774/References"

H37rv_Ref_FA_PATH = f"{Mtb_RefDir}/GCF_000195955.2_ASM19595v2_genomic.fasta"
IS6110_Example_FA_PATH = f"{Mtb_RefDir}/IS6110_From_Rv0795_Rv0796.DNA.fasta"

#### H37Rv - k-mer generation & hashing

In [34]:
H37Rv_kmers = read_kmers_from_file(H37rv_Ref_FA_PATH, k_size)

H37Rv_Hashes_Set = hash_kmers_ToSet(H37Rv_kmers)

print(len(H37Rv_kmers))

4411502


#### IS6110 (Rv0795 & Rv0796) - k-mer generation & hashing

In [35]:
IS6110_Ex1_kmers = read_kmers_from_file(IS6110_Example_FA_PATH, k_size)

IS6110_Ex1_Hashes_Set = hash_kmers_ToSet(IS6110_Ex1_kmers)

print(len(IS6110_Ex1_kmers))

1254


## Generate k-mer info for all H37Rv gene DNA sequences (Mycobrowser)

In [36]:
O2_RefDir = "/n/data1/hms/dbmi/farhat/mm774/References"
MycoBrowser_RefFiles_Dir = f"{O2_RefDir}/190619_Mycobrowser_H37rv_ReferenceFiles"

H37Rv_Genes_MycoBro_FA = f"{MycoBrowser_RefFiles_Dir}/Mycobacterium_tuberculosis_H37Rv_genes_v3.fasta"


In [37]:
!grep ^">" $H37Rv_Genes_MycoBro_FA | grep "dnaA"

>Rv0001|dnaA|CDS|1-1524|+|Chromosomal replication initiator protein DnaA


### Get 31-mer hashes for all annotated gene DNA sequences

In [38]:
dictOf_H37Rv_MycoBrow_GeneSeq = {}
dictOf_H37Rv_MycoBrow_Gene_KmerHashes = {}

for index, record in tqdm(enumerate(SeqIO.parse(H37Rv_Genes_MycoBro_FA, "fasta"))):
    
    RecordName = record.name
    RvID = RecordName.split("|")[0]
    GeneID = RecordName.split("|")[1]
    S_Seq = str(record.seq).upper()
    
    dictOf_H37Rv_MycoBrow_GeneSeq[GeneID] = S_Seq

    record_Hashes_Set = hash_kmers_ToSet(build_kmers(S_Seq, k_size))

    dictOf_H37Rv_MycoBrow_Gene_KmerHashes[GeneID] = record_Hashes_Set
    

4187it [00:24, 173.96it/s]


In [39]:
len(dictOf_H37Rv_MycoBrow_GeneSeq["dnaA"])

1524

In [40]:
list(dictOf_H37Rv_MycoBrow_Gene_KmerHashes["dnaA"])[:2]

[13580233940393664509, 5138456728421695490]

## Generate k-mer info for "Phage Sequences & Insertion Sequences) in H37Rv

## a) Create DF for only MGEs (Insertion seqs & phages)

In [41]:
H37Rv_Anno_InsSeqAndPhages_DF = H37Rv_GenomeAnno_Genes_DF.query("Functional_Category == 'insertion seqs and phages'")     

InsSeqAndPhages_GeneIDs = H37Rv_Anno_InsSeqAndPhages_DF["Symbol"].values
len(InsSeqAndPhages_GeneIDs)
     

147

In [42]:
#### Peak at the first 5 geneIDs

In [43]:
InsSeqAndPhages_GeneIDs[:5]

array(['Rv0031', 'Rv0094c', 'Rv0095c', 'Rv0336', 'Rv0393'], dtype=object)

In [44]:
H37Rv_Anno_InsSeqAndPhages_DF.head(1)

,Chrom,Start,End,Strand,H37rv_GeneID,Symbol,Feature,Functional_Category,Is_Pseudogene,Product,PEandPPE_Subfamily,ExcludedGroup_Category
33,NC_000962.3,33581,33794,+,Rv0031,Rv0031,CDS,insertion seqs and phages,No,Possible remnant of a transposase,None,InsertionSeqs_And_Phages


## b) Generate union of all 31 bp k-mer hashes for "Insertion seqs & phages" genes (N=147)

In [45]:
def getAllHashes_InTargetSeqs(dictOfHashes, targetsToKeep):
    
    all_Hashes_InTarget = set()

    for i_SeqID, i_Hashes in dictOfHashes.items():
                
        if i_SeqID in targetsToKeep:
            all_Hashes_InTarget.update(i_Hashes) 
    
    return all_Hashes_InTarget

In [46]:
Rv_MGEs_Hashes_Set = getAllHashes_InTargetSeqs(dictOf_H37Rv_MycoBrow_Gene_KmerHashes,
                                                    InsSeqAndPhages_GeneIDs)   

print(len(Rv_MGEs_Hashes_Set))

69102


#### Look at Jaccard Containment between 31-mers of H37Rv and H37Rv's MGEs (ISs + Phages)

In [47]:
jaccard_containment_FromSets(H37Rv_Hashes_Set, Rv_MGEs_Hashes_Set)

0.01589562466616704

# Part 3: Generate k-mer info per node from Pangraph GFA 

## Define output dir of the Mtb-WGA-SMK processing pipeline

In [48]:
WGA_SMK_Outputs_Dir = "/n/data1/hms/dbmi/farhat/mm774/Projects/Mtb-WGA-SMK-Output"

WGA151CI_SMK_OutputDir = WGA_SMK_Outputs_Dir + "/231121_MtbSetV3_151CI"
MtbWGA_SMK_Pipeline_OutputDir = WGA151CI_SMK_OutputDir


### Define paths to processed analysis of SV Pan-genome graph (minigraph)

In [49]:
PangenomeAnalysis_Dir = "../../Data/MtbPangenomeAnalysis_SetV5"

MG_Node_KmerComp_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Minigraph.NodeKmerComp.Summary.V1.tsv.gz" 

MG_AvA_Node_KmerAnalysis_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Minigraph.NodeKmerComp.AllVsAll.V1.tsv.gz"     

MG_BubbleSumm_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Minigraph.BubbleSummary.BED.tsv.gz"     

MG_SVVCF_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Minigraph.BubbleAlleleInfo.SVVCF.tsv.gz" 

MG_SVInfo_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Minigraph.BubbleAlleleInfo.SVInfo.tsv.gz" 


### Parse in `MG_Nodes_KmerVsRefSets_DF`

In [50]:
MG_Nodes_KmerComp_DF = pd.read_csv(MG_Node_KmerComp_TSV_GZ, sep = "\t" )
MG_Nodes_KmerComp_DF.shape

(3138, 18)

In [51]:
MG_Nodes_KmerComp_DF.head()

,NodeID,IsSVNode,SeqLength,Jaccard_Cont_WiRv,Jaccard_Cont_WiIS6110,Jaccard_Cont_WiRv_InsSeqAndPhages,Jaccard_Cont_WiRv_PEPPEs,Jaccard_Cont_WiRv_InfoPathways,Jaccard_Cont_WiRv_ConservedHypo,Jaccard_Cont_WiRv_CellWallCellProc,Jaccard_Cont_WiRv_StableRNAs,Jaccard_Cont_WiRv_InterMetabolism,Jaccard_Cont_WiRv_RegProteins,Jaccard_Cont_WiRv_VirulenceDetoxAdaptation,Jaccard_Cont_WiRv_LipidMetabolism,Jaccard_Cont_WiRv_Unknown,BubbleID,MaxJC_ToOtherNode
0,s1,False,1533,1.0,0.0,0.0,0.0,0.994012,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,None,0.000000
1,s2,False,58,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,None,0.000000
2,s3,True,56,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,BubbleRegion_2,0.961538
3,s4,True,5,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,BubbleRegion_2,0.000000
4,s5,False,11970,1.0,0.0,0.0,0.0,0.612312,0.044724,0.141457,0.007286,0.0,0.0,0.0,0.0,0.0,None,0.000000


# Define path to outputs of Pangraph (3 different parameters)

In [52]:
#!ls -1 $MtbWGA_SMK_Pipeline_OutputDir

In [53]:
target_OutputDir = MtbWGA_SMK_Pipeline_OutputDir

# Minigraph_151CI_OutDir = f"{target_OutputDir}/Minigraph"

Pangraph_MainOutDir = f"{target_OutputDir}/Pangraph_Mtb151CIWiRv_Analysis"

Pangraph_ParamSet1_Out = f"{Pangraph_MainOutDir}/Pangraph_ParamSet1_RvWithMtb151CI"
Pangraph_PS1_FA        = f"{Pangraph_ParamSet1_Out}/pangraph.fa"
Pangraph_PS1_NoMod_GFA = f"{Pangraph_ParamSet1_Out}/pangraph.gfa"
Pangraph_PS1_GFA       = f"{Pangraph_ParamSet1_Out}/pangraph.CigarMod.gfa"

Pangraph_ParamSet2_Out = f"{Pangraph_MainOutDir}/Pangraph_ParamSet2_RvWithMtb151CI"
Pangraph_PS2_FA        = f"{Pangraph_ParamSet2_Out}/pangraph.fa"
Pangraph_PS2_NoMod_GFA = f"{Pangraph_ParamSet2_Out}/pangraph.gfa"
Pangraph_PS2_GFA       = f"{Pangraph_ParamSet2_Out}/pangraph.CigarMod.gfa"

Pangraph_ParamSet3_Out = f"{Pangraph_MainOutDir}/Pangraph_ParamSet3_RvWithMtb151CI"
Pangraph_PS3_FA        = f"{Pangraph_ParamSet3_Out}/pangraph.fa"
Pangraph_PS3_NoMod_GFA = f"{Pangraph_ParamSet3_Out}/pangraph.gfa"
Pangraph_PS3_GFA       = f"{Pangraph_ParamSet3_Out}/pangraph.CigarMod.gfa"


In [54]:
!ls -1 $Pangraph_MainOutDir

OLD_Pangraph_V1
Pangraph_151CI_V3_NoDup
Pangraph_151CI_V3_WiDup
Pangraph_151CI_V4_WiDup
Pangraph.Commands.txt
Pangraph_ParamSet1_RvWithMtb151CI
Pangraph_ParamSet2_RvWithMtb151CI
Pangraph_ParamSet3_RvWithMtb151CI


#### Replace cigar string in Pangraph GFAs to make it compatible with GFA parser

In [55]:
sed_replacecigar_string = 's/TP:Z:circular//'

!sed -E $sed_replacecigar_string $Pangraph_PS1_NoMod_GFA > $Pangraph_PS1_GFA
!sed -E $sed_replacecigar_string $Pangraph_PS2_NoMod_GFA > $Pangraph_PS2_GFA
!sed -E $sed_replacecigar_string $Pangraph_PS3_NoMod_GFA > $Pangraph_PS3_GFA


In [56]:
!ls -lah $Pangraph_PS1_NoMod_GFA

-rw-r--r-- 1 mm774 farhat 1.8M Dec 27 11:57 /n/data1/hms/dbmi/farhat/mm774/Projects/Mtb-WGA-SMK-Output/231121_MtbSetV3_151CI/Pangraph_Mtb151CIWiRv_Analysis/Pangraph_ParamSet1_RvWithMtb151CI/pangraph.gfa


In [57]:
!ls -lah $Pangraph_PS1_GFA

-rw-r--r-- 1 mm774 farhat 1.8M Dec 28 14:50 /n/data1/hms/dbmi/farhat/mm774/Projects/Mtb-WGA-SMK-Output/231121_MtbSetV3_151CI/Pangraph_Mtb151CIWiRv_Analysis/Pangraph_ParamSet1_RvWithMtb151CI/pangraph.CigarMod.gfa


In [58]:
!ls -lah $Pangraph_PS2_GFA

-rw-r--r-- 1 mm774 farhat 2.1M Dec 28 14:50 /n/data1/hms/dbmi/farhat/mm774/Projects/Mtb-WGA-SMK-Output/231121_MtbSetV3_151CI/Pangraph_Mtb151CIWiRv_Analysis/Pangraph_ParamSet2_RvWithMtb151CI/pangraph.CigarMod.gfa


In [59]:
!ls -lah $Pangraph_PS3_GFA

-rw-r--r-- 1 mm774 farhat 1.9M Dec 28 14:50 /n/data1/hms/dbmi/farhat/mm774/Projects/Mtb-WGA-SMK-Output/231121_MtbSetV3_151CI/Pangraph_Mtb151CIWiRv_Analysis/Pangraph_ParamSet3_RvWithMtb151CI/pangraph.CigarMod.gfa


# Pangraph data processing

In [60]:
# Define predefined k-mer sets
Rv_ref_kmer_sets = {
    "H37Rv": H37Rv_Hashes_Set,
    "IS6110": IS6110_Ex1_Hashes_Set,
    "Rv_InsSeqAndPhages": Rv_MGEs_Hashes_Set
}

k_size = 31
num_assemblies = 152

## Process Pangraph - ParamSet 1

In [61]:
# Step 1: Parse FASTA
PS1_node_seqs = parse_pangraph_fasta(Pangraph_PS1_FA)

# Step 2: Parse GFA
PS1_node_info = parse_pangraph_gfa(Pangraph_PS1_GFA,
                                   PS1_node_seqs, k_size, num_assemblies)

# Step 3: Classify nodes
PS1_core_nodes, PS1_sv_nodes = classify_pangraph_nodes(PS1_node_info, num_assemblies)

print("# of Core Nodes:", len(PS1_core_nodes))
print("# of SV Nodes:", len(PS1_sv_nodes))
print("# of All Nodes:", len(PS1_node_info))
print()

PS1_AvA_DF = pangraph_pairwise_kmerset_comparison(PS1_node_info)

PS1_NodeSummary_DF = compare_kmers_to_profiles_with_max_jc(PS1_node_info,
                                                          PS1_sv_nodes,
                                                          Rv_ref_kmer_sets,
                                                          PS1_AvA_DF)

print("Dimensions of final node summary DF:", PS1_NodeSummary_DF.shape)

Parsing FASTA: 1341it [00:00, 18724.88it/s]
Running pairwise comparisons of k-mer content:   0%|          | 3/1341 [00:00<00:59, 22.32it/s]

# of Core Nodes: 496
# of SV Nodes: 845
# of All Nodes: 1341



Comparing node to reference k-mer profiles: 100%|██████████| 1341/1341 [00:01<00:00, 1157.21it/s]

Dimensions of final node summary DF: (1341, 9)


## Process Pangraph - ParamSet 2

In [62]:
# Step 1: Parse FASTA
PS2_node_seqs = parse_pangraph_fasta(Pangraph_PS2_FA)

# Step 2: Parse GFA
PS2_node_info = parse_pangraph_gfa(Pangraph_PS2_GFA,
                                   PS2_node_seqs, k_size, num_assemblies)

# Step 3: Classify nodes
PS2_core_nodes, PS2_sv_nodes = classify_pangraph_nodes(PS2_node_info, num_assemblies)

print("# of Core Nodes:", len(PS2_core_nodes))
print("# of SV Nodes:", len(PS2_sv_nodes))
print("# of All Nodes:", len(PS2_node_info))
print()

PS2_AvA_DF = pangraph_pairwise_kmerset_comparison(PS2_node_info)

PS2_NodeSummary_DF = compare_kmers_to_profiles_with_max_jc(PS2_node_info,
                                                           PS2_sv_nodes,
                                                           Rv_ref_kmer_sets,
                                                           PS2_AvA_DF)

print("Dimensions of final node summary DF:", PS2_NodeSummary_DF.shape)

Parsing FASTA: 1724it [00:00, 39803.27it/s]
Running pairwise comparisons of k-mer content:   0%|          | 4/1724 [00:00<00:49, 34.58it/s]

# of Core Nodes: 555
# of SV Nodes: 1169
# of All Nodes: 1724



Comparing node to reference k-mer profiles: 100%|██████████| 1724/1724 [00:01<00:00, 1549.80it/s]

Dimensions of final node summary DF: (1724, 9)


## Process Pangraph - ParamSet 3

In [63]:
# Step 1: Parse FASTA
PS3_node_seqs = parse_pangraph_fasta(Pangraph_PS3_FA)

# Step 2: Parse GFA
PS3_node_info = parse_pangraph_gfa(Pangraph_PS3_GFA,
                                   PS3_node_seqs, k_size, num_assemblies)

# Step 3: Classify nodes
PS3_core_nodes, PS3_sv_nodes = classify_pangraph_nodes(PS3_node_info, num_assemblies)

print("# of Core Nodes:", len(PS3_core_nodes))
print("# of SV Nodes:", len(PS3_sv_nodes))
print("# of All Nodes:", len(PS3_node_info))
print()

PS3_AvA_DF = pangraph_pairwise_kmerset_comparison(PS3_node_info)

PS3_NodeSummary_DF = compare_kmers_to_profiles_with_max_jc(PS3_node_info,
                                                           PS3_sv_nodes,
                                                           Rv_ref_kmer_sets,
                                                           PS3_AvA_DF)

print("Dimensions of final node summary DF:", PS3_NodeSummary_DF.shape)

Parsing FASTA: 1481it [00:00, 35733.47it/s]
Running pairwise comparisons of k-mer content:   0%|          | 4/1481 [00:00<00:45, 32.64it/s]

# of Core Nodes: 524
# of SV Nodes: 957
# of All Nodes: 1481



Comparing node to reference k-mer profiles: 100%|██████████| 1481/1481 [00:01<00:00, 1205.04it/s]

Dimensions of final node summary DF: (1481, 9)


# Output node summaries for exploration

In [67]:
PangenomeAnalysis_Dir = "../../Data/MtbPangenomeAnalysis_SetV5"

PG_PS1_Node_Summary_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Pangraph.ParamSet1.NodeKmerComp.Summary.V1.tsv.gz" 
PG_PS1_AvA_NodeComp_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Pangraph.ParamSet1.NodeKmerComp.AllVsAll.V1.tsv.gz"     

PG_PS2_Node_Summary_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Pangraph.ParamSet2.NodeKmerComp.Summary.V1.tsv.gz" 
PG_PS2_AvA_NodeComp_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Pangraph.ParamSet2.NodeKmerComp.AllVsAll.V1.tsv.gz"     

PG_PS3_Node_Summary_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Pangraph.ParamSet3.NodeKmerComp.Summary.V1.tsv.gz" 
PG_PS3_AvA_NodeComp_TSV_GZ = f"{PangenomeAnalysis_Dir}/MtbSVPG.Pangraph.ParamSet3.NodeKmerComp.AllVsAll.V1.tsv.gz"     



PS1_AvA_DF.to_csv(PG_PS1_AvA_NodeComp_TSV_GZ,
                  sep = "\t", index=False)

PS1_NodeSummary_DF.to_csv(PG_PS1_Node_Summary_TSV_GZ,
                          sep = "\t", index=False)

PS2_AvA_DF.to_csv(PG_PS2_AvA_NodeComp_TSV_GZ,
                  sep = "\t", index=False)

PS2_NodeSummary_DF.to_csv(PG_PS2_Node_Summary_TSV_GZ,
                          sep = "\t", index=False)


PS3_AvA_DF.to_csv(PG_PS3_AvA_NodeComp_TSV_GZ,
                  sep = "\t", index=False)

PS3_NodeSummary_DF.to_csv(PG_PS3_Node_Summary_TSV_GZ,
                          sep = "\t", index=False)





In [68]:
!ls -lah $PangenomeAnalysis_Dir

total 2.3M
drwxr-sr-x 2 mm774 farhat 4.0K Dec 28 14:59 .
drwxrwsr-x 2 mm774 farhat 4.0K Dec  6 15:23 ..
-rw-r--r-- 1 mm774 farhat 2.8K Nov 25 21:33 MtbSVPG.BubbleCoords.1kbUnqToRv.18B.tsv
-rw-r--r-- 1 mm774 farhat 1.5K Nov 25 21:33 MtbSVPG.BubbleStats.UnqKmerContent.1kbUnqToRv.18B.tsv
-rw-r--r-- 1 mm774 farhat  16K Nov 25 21:33 MtbSVPG.BubbleStats.UnqKmerContent.All.532B.tsv
-rw-r--r-- 1 mm774 farhat 631K Nov 25 16:39 MtbSVPG.Minigraph.BubbleAlleleInfo.SVInfo.tsv.gz
-rw-r--r-- 1 mm774 farhat  61K Nov 25 16:39 MtbSVPG.Minigraph.BubbleAlleleInfo.SVVCF.tsv.gz
-rw-r--r-- 1 mm774 farhat  32K Nov 25 16:35 MtbSVPG.Minigraph.BubbleSummary.BED.tsv.gz
-rw-r--r-- 1 mm774 farhat 1.2M Nov 25 16:38 MtbSVPG.Minigraph.NodeKmerComp.AllVsAll.V1.tsv.gz
-rw-r--r-- 1 mm774 farhat  57K Nov 25 16:39 MtbSVPG.Minigraph.NodeKmerComp.Summary.V1.tsv.gz
-rw-r--r-- 1 mm774 farhat  77K Nov 26 09:36 MtbSVPG.Pangraph.NodeKmerComp.AllVsAll.V1.tsv.gz
-rw-r--r-- 1 mm774 farhat  28K Nov 26 09:36 MtbSVPG.Pangraph.NodeKmerC

# Summary Analysis - Round 2

In [113]:
# Summarize statistics
summary_stats = summarize_Pangenome_Node_Stats(PS1_NodeSummary_DF)

# Display summary
for key, value in summary_stats.items():
    
    print(f"{key}: {value}")

N_CoreNodes: 496
SeqLength_CoreNodes: 3971389
N_SV_All: 845
SeqLength_SV_All: 465526
N_SV_UnqInGraph: 579
SeqLength_SV_UnqInGraph: 410195
N_SV_RepeatInGraph: 265
SeqLength_SV_RepeatInGraph: 55312
N_SV_UnqToH37Rv: 95
SeqLength_SV_UnqToH37Rv: 68434


In [110]:
# Summarize statistics
summary_stats = summarize_Pangenome_Node_Stats(PS2_NodeSummary_DF)

# Display summary
for key, value in summary_stats.items():
    
    print(f"{key}: {value}")

N_CoreNodes: 555
SeqLength_CoreNodes: 3941145
N_SV_All: 1169
SeqLength_SV_All: 481565
N_SV_UnqInGraph: 717
SeqLength_SV_UnqInGraph: 417213
N_SV_RepeatInGraph: 450
SeqLength_SV_RepeatInGraph: 64307


In [111]:
# Summarize statistics
summary_stats = summarize_Pangenome_Node_Stats(PS3_NodeSummary_DF)

# Display summary
for key, value in summary_stats.items():
    
    print(f"{key}: {value}")

N_CoreNodes: 524
SeqLength_CoreNodes: 3964632
N_SV_All: 957
SeqLength_SV_All: 468998
N_SV_UnqInGraph: 630
SeqLength_SV_UnqInGraph: 419457
N_SV_RepeatInGraph: 326
SeqLength_SV_RepeatInGraph: 49522


In [117]:
summary_stats_Dict = {"Minigraph"   : summarize_Pangenome_Node_Stats(MG_Nodes_KmerComp_DF), 
                      "Pangraph_PS1" : summarize_Pangenome_Node_Stats(PS1_NodeSummary_DF), 
                      "Pangraph_PS2" : summarize_Pangenome_Node_Stats(PS2_NodeSummary_DF),  
                      "Pangraph_PS3" : summarize_Pangenome_Node_Stats(PS3_NodeSummary_DF) }

SummStats_DF_List = []

for key, i_Dict in summary_stats_Dict.items():
    i_Dict["Method"] = key

    i_DF = pd.DataFrame([i_Dict])
    
    SummStats_DF_List.append(i_DF)
    


All_Stats_DF = pd.concat(SummStats_DF_List, axis = 0)

# Move the "Method" column to the first position
method_col = All_Stats_DF.pop("Method")

All_Stats_DF.insert(0, "Method", method_col)

All_Stats_DF.shape

(4, 11)

In [118]:
All_Stats_DF

,Method,N_CoreNodes,SeqLength_CoreNodes,N_SV_All,SeqLength_SV_All,N_SV_UnqInGraph,SeqLength_SV_UnqInGraph,N_SV_RepeatInGraph,SeqLength_SV_RepeatInGraph,N_SV_UnqToH37Rv,SeqLength_SV_UnqToH37Rv
0,Minigraph,536,3913140,2602,1283223,463,301511,1562,975039,160,66621
0,Pangraph_PS1,496,3971389,845,465526,579,410195,265,55312,95,68434
0,Pangraph_PS2,555,3941145,1169,481565,717,417213,450,64307,133,70509
0,Pangraph_PS3,524,3964632,957,468998,630,419457,326,49522,102,69058


### Produce Sankey plots for the different approaches
Using [https://sankeymatic.com/build/](https://sankeymatic.com/build/)

### Input code for sankeymatic

#### a) SV Graph (minigraph)
```
SV Graph (minigraph) [3913140] Core nodes #808080
SV Graph (minigraph) [1283223] SV nodes #808080
SV nodes [981712] Non-unique Nodes #808080
SV nodes [301511] Unique Nodes #808080

// You can set a Node's color, like this:
:SV Graph (minigraph) #A020F0
:SV nodes #4D4E75
:Core nodes #BD555A
:Non-unique Nodes #505D96
:Unique Nodes #51ACA6
```


#### b) SV Graph (Pangraph Parameter Set #1)
```
SV Graph (Pangraph PS #1) [3971389] Core nodes #808080
SV Graph (Pangraph PS #1) [465526] SV nodes #808080
SV nodes [55331] Non-unique Nodes #808080
SV nodes [410195] Unique Nodes #808080

// You can set a Node's color, like this:
:SV Graph (Pangraph PS #1) #A020F0
:SV nodes #4D4E75
:Core nodes #BD555A
:Non-unique Nodes #505D96
:Unique Nodes #51ACA6
```


#### c) SV Graph (Pangraph Parameter Set #2)
```
SV Graph (Pangraph PS #2) [3941145] Core nodes #808080
SV Graph (Pangraph PS #2) [481565] SV nodes #808080
SV nodes [64352] Non-unique Nodes #808080
SV nodes [417213] Unique Nodes #808080

// You can set a Node's color, like this:
:SV Graph (Pangraph PS #2) #A020F0
:SV nodes #4D4E75
:Core nodes #BD555A
:Non-unique Nodes #505D96
:Unique Nodes #51ACA6
```


#### c) SV Graph (Pangraph Parameter Set #3)
```
SV Graph (Pangraph PS #3) [3964632] Core nodes #808080
SV Graph (Pangraph PS #3) [468998] SV nodes #808080
SV nodes [49541] Non-unique Nodes #808080
SV nodes [419457] Unique Nodes #808080

// You can set a Node's color, like this:
:SV Graph (Pangraph PS #3) #A020F0
:SV nodes #4D4E75
:Core nodes #BD555A
:Non-unique Nodes #505D96
:Unique Nodes #51ACA6
```